## Make Graphs from Meshes (for testing)

Move this to archived soon. Mostly still for debugging but should all be in `PretrainFromYCBMeshes` experiment class.

In [ ]:
import os
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.data import Data

from tbp.monty.frameworks.environments.ycb import YCBMeshDataset
from tbp.monty.frameworks.utils.plot_utils import plot_graph
from tbp.monty.frameworks.environment_utils.graph_utils import get_point_normal, get_curvature_at_point, get_graph_from_mesh

In [ ]:
%matplotlib notebook

In [ ]:
ycb_data = YCBMeshDataset(
            os.path.expanduser("~/tbp/data/habitat/objects/ycb")
        )
graph = ycb_data.get_mesh_by_name('potted_meat_can')
graph = get_graph_from_mesh(graph, simplify=3000)

graph = T.Compose(
    [
        T.FaceToEdge(remove_faces=False),
        T.GenerateMeshNormals(),
    ]
)(graph)
graph.pos = graph.pos * torch.tensor([-1, -1, 1])

In [ ]:
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
pos = np.array(graph.pos)
s = ax.scatter(
    pos[:, 0],
    pos[:, 1],
    pos[:, 2],
    s=5,
)
norm_len = 0.01
for p_id, p in enumerate(pos):
    norm = graph.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='lightgreen',
           )
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
pos = np.array(graph.pos)
s = ax.scatter(
    pos[:, 0],
    pos[:, 1],
    pos[:, 2],
    s=5,
)
norm_len = 0.01
for p_id, p in enumerate(pos):
    norm = graph.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='lightgreen',
           )
plt.show()

In [ ]:
graph.face

In [ ]:
# Old, do this instead with python  projects/monty_runs/run.py -e supervised_pre_training_on_mesh
def get_angle(vec1, vec2):
        # TODO: move to helper functions
        unit_vector_1 = vec1 / np.linalg.norm(vec1)
        unit_vector_2 = vec2 / np.linalg.norm(vec2)
        dot_product = np.dot(unit_vector_1, unit_vector_2)
        angle = np.arccos(np.clip(dot_product, -1, 1))
        return angle

def build_pretrain_models_from_meshes(objects, save_path, radius=0.005, simplify_mesh=10000, subsample_graph=None):
    ycb_data = YCBMeshDataset(os.path.expanduser("~/tbp/data/habitat/objects/ycb"))
    state_dict = {"lm_dict": {0:{"graph_memory":dict()}}}
    for obj in objects:
        graph = ycb_data.get_mesh_by_name(obj)
        graph = get_graph_from_mesh(graph, simplify=simplify_mesh)
        graph.pos = graph.pos * torch.tensor([-1, -1, 1])
        graph = T.Compose([
                    T.FaceToEdge(remove_faces=False),
                        T.GenerateMeshNormals(),
#                         T.PointPairFeatures(cat=True),
                    ])(graph)
        print(graph)
        print("got graph from mesh")
        
        point_cloud = np.array(graph.pos)
#         print(f"whole point cloud: {point_cloud.shape}")
        curvatures = []
        for p_id, p in enumerate(point_cloud):
            distances_to_point = np.linalg.norm(np.array(graph.pos) - p, axis=1)
        #             print(f"distances: {distances_to_point.shape} with min {np.min(distances_to_point)} and max {np.max(distances_to_point)}")
            radius_ids = np.where(distances_to_point <= radius)[0]
            radius_point_cloud = point_cloud[radius_ids]
            radius_norms = graph.norm[radius_ids]
            angles = []
            for n in radius_norms:
                a = get_angle(graph.norm[p_id], n)
                angles.append(a)
            angles = np.array(angles)
            radius_angle_ids = np.where(angles < np.pi)[0]
            radius_angle_pc = radius_point_cloud[radius_angle_ids]
            pc_angles = angles[radius_angle_ids]
            new_p_id = np.argmin(pc_angles)
            radius_angle_pc = np.hstack([radius_angle_pc, np.ones((radius_angle_pc.shape[0],1))])
            curvature = get_curvature_at_point(radius_angle_pc, new_p_id, graph.norm[p_id])
            curvatures.append(np.array(curvature[:2]))
        print("extracted curvatures")
        node_ids = np.linspace(0,graph.pos.shape[0]-1,graph.pos.shape[0])
        features = np.hstack([node_ids[:,np.newaxis], 
                                  np.array(graph.norm),
                                     np.array(curvatures)])
        feature_mapping = {'node_ids': [0,1], 
                           'point_normal':[1,4],
                           'curvature': [4, 6]}
        graph.x = torch.tensor(features, dtype=torch.float)
        graph.feature_mapping = feature_mapping
        if subsample_graph is not None:
            sp = np.random.randint(0, graph.pos.shape[0], subsample_graph)
            new_x = graph.x[sp]
            new_x[:,0] = torch.tensor(np.linspace(0,subsample_graph-1,subsample_graph), dtype=torch.float)
            graph = Data(x=new_x, pos=graph.pos[sp], norm=graph.norm[sp], feature_mapping=graph.feature_mapping)
        print("added features to graph")
        state_dict["lm_dict"][0]["graph_memory"][obj] = graph
        if "target_to_graph_id" not in state_dict["lm_dict"][0].keys():
            state_dict["lm_dict"][0]["target_to_graph_id"] = dict()
        state_dict["lm_dict"][0]["target_to_graph_id"][obj] = {obj}
        if "graph_id_to_target" not in state_dict["lm_dict"][0].keys():
            state_dict["lm_dict"][0]["graph_id_to_target"] = dict()
        state_dict["lm_dict"][0]["graph_id_to_target"][obj] = {obj}
#     graph_dir = save_path + "pretrained/"
#     os.makedirs(graph_dir, exist_ok=True)
#     torch.save(state_dict, graph_dir + "model.pt")
#     print("saved models")
    return graph, radius_point_cloud, radius_norms, radius_angle_pc, new_p_id, graph.norm[p_id] # just return the last graph for plotting/debugging

In [ ]:
# Old, do this instead with python  projects/monty_runs/run.py -e supervised_pre_training_on_mesh
def get_angle(vec1, vec2):
        # TODO: move to helper functions
        unit_vector_1 = vec1 / np.linalg.norm(vec1)
        unit_vector_2 = vec2 / np.linalg.norm(vec2)
        dot_product = np.dot(unit_vector_1, unit_vector_2)
        angle = np.arccos(np.clip(dot_product, -1, 1))
        return angle

def build_pretrain_models_from_meshes(objects, save_path, radius=0.005, simplify_mesh=10000, subsample_graph=None):
    ycb_data = YCBMeshDataset(os.path.expanduser("~/tbp/data/habitat/objects/ycb"))
    state_dict = {"lm_dict": {0:{"graph_memory":dict()}}}
    for obj in objects:
        graph = ycb_data.get_mesh_by_name(obj)
        graph = get_graph_from_mesh(graph, simplify=simplify_mesh)
        graph = T.Compose([
                    T.FaceToEdge(remove_faces=False),
                        T.GenerateMeshNormals(),
                        T.PointPairFeatures(cat=True),
                    ])(graph)
        print(graph)
        print("got graph from mesh")
        
        point_cloud = np.array(graph.pos)
#         print(f"whole point cloud: {point_cloud.shape}")
        curvatures = []
        for p_id, p in enumerate(point_cloud):
            if np.linalg.norm(np.array(graph.norm[p_id])) < 0.9:
                print(f"norm for point {p_id} has invalid norm {graph.norm[p_id]}.")
                curvatures.append(np.array([0,0]))
            else:
                distances_to_point = np.linalg.norm(np.array(graph.pos) - p, axis=1)
            #             print(f"distances: {distances_to_point.shape} with min {np.min(distances_to_point)} and max {np.max(distances_to_point)}")
                radius_ids = np.where(distances_to_point <= radius)[0]
                radius_point_cloud = point_cloud[radius_ids]
                radius_norms = graph.norm[radius_ids]
                angles = []
                for n in radius_norms:
                    a = get_angle(graph.norm[p_id], n)
                    angles.append(a)
                angles = np.array(angles)
                radius_angle_ids = np.where(angles < np.pi/2)[0]
                radius_angle_pc = radius_point_cloud[radius_angle_ids]
                pc_angles = angles[radius_angle_ids]
                new_p_id = np.argmin(pc_angles)
                radius_angle_pc = np.hstack([radius_angle_pc, np.ones((radius_angle_pc.shape[0],1))])
                curvature = get_curvature_at_point(radius_angle_pc, new_p_id, graph.norm[p_id])
                curvatures.append(np.array(curvature[:2]))
        print("extracted curvatures")
        node_ids = np.linspace(0,graph.pos.shape[0]-1,graph.pos.shape[0])
        features = np.hstack([node_ids[:,np.newaxis], 
                                  np.array(graph.norm),
                                     np.array(curvatures)])
        feature_mapping = {'node_ids': [0,1], 
                           'point_normal':[1,4],
                           'curvature': [4, 6]}
        graph.x = torch.tensor(features, dtype=torch.float)
        graph.feature_mapping = feature_mapping
        if subsample_graph is not None:
            sp = np.random.randint(0, graph.pos.shape[0], subsample_graph)
            new_x = graph.x[sp]
            new_x[:,0] = torch.tensor(np.linspace(0,subsample_graph-1,subsample_graph), dtype=torch.float)
            graph = Data(x=new_x, pos=graph.pos[sp], norm=graph.norm[sp], feature_mapping=graph.feature_mapping)
        print("added features to graph")
        state_dict["lm_dict"][0]["graph_memory"][obj] = graph
        if "target_to_graph_id" not in state_dict["lm_dict"][0].keys():
            state_dict["lm_dict"][0]["target_to_graph_id"] = dict()
        state_dict["lm_dict"][0]["target_to_graph_id"][obj] = {obj}
        if "graph_id_to_target" not in state_dict["lm_dict"][0].keys():
            state_dict["lm_dict"][0]["graph_id_to_target"] = dict()
        state_dict["lm_dict"][0]["graph_id_to_target"][obj] = {obj}
#     graph_dir = save_path + "pretrained/"
#     os.makedirs(graph_dir, exist_ok=True)
#     torch.save(state_dict, graph_dir + "model.pt")
#     print("saved models")
    return graph#, radius_point_cloud, radius_norms, radius_angle_pc, new_p_id, graph.norm[p_id] # just return the last graph for plotting/debugging

In [ ]:
save_dir = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/mesh_models_sub_sampled/")
objects_to_learn = ["potted_meat_can"]#, "bowl","mug","potted_meat_can","master_chef_can"]
# graph = build_pretrain_models_from_meshes(["mug"], save_dir, radius=0.01, simplify=10000)
#, pc, rn, pcr, pid, pnr
graph = build_pretrain_models_from_meshes(objects_to_learn, 
                                          save_dir, 
                                          radius=0.02, 
                                          simplify_mesh=0,
                                        subsample_graph=None)


In [ ]:
# Plot radius patch (uncomment line above) for debugging
"""fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
model_pos = pc

# s = ax.scatter(
#     model_pos[:, 0],
#     model_pos[:, 1],
#     model_pos[:, 2],
#     s=8,
#     c='lightblue',
#     alpha=0.9
# )
s2 = ax.scatter(
    pcr[:, 0],
    pcr[:, 1],
    pcr[:, 2],
    s=5,
    c='green'
)
s3 = ax.scatter(
    pcr[pid, 0],
    pcr[pid, 1],
    pcr[pid, 2],
    s=20,
    c='red'
)
norm_len=0.01
p = pcr[pid]
ax.plot([p[0], p[0] + pnr[0] * norm_len],
            [p[1], p[1] + pnr[1] * norm_len],
            [p[2], p[2] + pnr[2] * norm_len],
            c='lightgreen',
           )
for p_id, p in enumerate(np.array(pc)):
    norm = rn[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='lightgreen',
           )
# ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
# ax.set_xlim(-0.1, 0.1)
# ax.set_ylim(-0.15,0.05)
# ax.set_zlim(-0.05, 0.15)
plt.show()"""

In [ ]:
from tbp.monty.frameworks.utils.sensor_processing import scale_clip

In [ ]:
subsample_graph = 3000
sp = np.random.randint(0, graph.pos.shape[0], subsample_graph)
new_x = graph.x[sp]
new_x[:,0] = torch.tensor(np.linspace(0,subsample_graph-1,subsample_graph), dtype=torch.float)
subs_graph = Data(x=new_x, pos=graph.pos[sp], norm=graph.norm[sp], feature_mapping=graph.feature_mapping)


In [ ]:
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
model_pos = subs_graph.pos
color = subs_graph.x[:,4] * subs_graph.x[:,5]
color = scale_clip(color, 4096)
# color = (graph.x[:,4] + graph.x[:,5]) /2
# color = scale_clip(color, 256)

s = ax.scatter(
    model_pos[:, 0],
    model_pos[:, 1],
    model_pos[:, 2],
    s=2,
    vmin=-64,
    vmax=64,
    cmap='seismic',
    c=color
)
# ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
ax.set_xlim(-0.1, 0.1)
ax.set_ylim(-0.15,0.05)
ax.set_zlim(-0.05, 0.15)
plt.title("Mean PC")
fig.colorbar(s)
plt.show()

### Compare Mesh Models to Sensor Models

In [ ]:
from tbp.monty.frameworks.utils.logging_utils import load_stats

In [ ]:
# General paths:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
log_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/")
# log_path = os.path.expanduser("~/tbp/results/monty/projects/feature_eval_runs/logs/")

# Specific experiment paths:
# exp_path = log_path + "feature_pred_tests/"
exp_path = log_path + "off_object_test/"
pretrained_dict_sensor = pretrain_path + "feature_eval_runs_xyz/pretrained_ycb4_stepsize5/supervised_pre_training_base/pretrained/"
pretrained_dict_mesh = pretrain_path + "feature_eval_runs_xyz/pretrained_models_from_mesh/ycb4_3000p/"


_, _, _, lm_models_mesh = load_stats(exp_path,
                                        load_train=False,
                                        load_eval=False,
                                        load_detailed=False,
                                        pretrained_dict=pretrained_dict_mesh,
                                       )
_, _, _, lm_models_sensor = load_stats(exp_path,
                                        load_train=False,
                                        load_eval=False,
                                        load_detailed=False,
                                        pretrained_dict=pretrained_dict_sensor,
                                       )

In [ ]:

plot_graph(lm_models_mesh['pretrained'][0]['mug'])
plt.show()

In [ ]:
plot_graph(lm_models_sensor['pretrained'][0]['mug'])
plt.show()

In [ ]:
sensorG = lm_models_sensor['pretrained'][0]['mug']
meshG = lm_models_mesh['pretrained'][0]['master_chef_can']

fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
# posS = np.array(sensorG.pos)
# s = ax.scatter(
#     posS[:, 0],
#     posS[:, 1],
#     posS[:, 2],
#     s=5,
#     color='blue'
# )

posM = np.array(meshG.pos)
s = ax.scatter(
    posM[:, 0],
    posM[:, 1],
    posM[:, 2],
    s=5,
    color='green'
)
norm_len = 0.01
for p_id, p in enumerate(posM):
    norm = meshG.norm[p_id]
    ax.plot([p[0], p[0] + norm[0] * norm_len],
            [p[1], p[1] + norm[1] * norm_len],
            [p[2], p[2] + norm[2] * norm_len],
            c='lightgreen',
           )
plt.show()